In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import collections
import os
import pathlib


pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


## Основные функции

In [ ]:
state_groups = pd.read_excel('/content/drive/MyDrive/Trade_CA/Сгруппированные_страны-YY.xlsx')
state_groups.tail()

,Страна,Группировка
224,Сан-Томе и Принсипи,Прочие
225,Вануату,Прочие
226,Палау,Прочие
227,Северные Марианские острова,Прочие
228,Микронезия (Федеративные Штаты),Прочие


In [ ]:
col_names_countries = list(range(2001,2023))
col_names_countries.insert(0,'Страна')
col_names_countries

['Страна',
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [ ]:
def change_colnames(df, cols):
  df.columns = cols
  return df

In [ ]:
# goods = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Импорт_ЦА_товары2знака.xlsx')
# print(goods.shape)
# print(len(col_names_goods))
# df_with_years = change_colnames(goods, col_names_goods)
# df_with_years

In [ ]:


def merge_columns(df, state_groups = state_groups):
  df_grouped = df.merge(state_groups, on = 'Страна').fillna(0)
  grouped_column = df_grouped.pop('Группировка')
  df_grouped.insert(loc=1, column='Группировка', value=grouped_column)
  return df_grouped


In [ ]:
# def flatten(x):
#   if isinstance(x, collections.abc.Iterable):
#       return [a for i in x for a in flatten(i)]
#   else:
#       return [x]

def sum_by_countriesgroup(df):
  df_group = df.melt(id_vars=["Страна", "Группировка"], var_name="Year", value_name="Value")
  df_group = df_group.replace(0, 1)
  df_group = df_group.pivot_table(values = 'Value', index = 'Группировка', columns= 'Year', aggfunc='sum').reset_index()
  df_group.loc['Total']= df_group.sum(numeric_only=True)
  df_group = df_group.reset_index(drop=True)
  df_group.iloc[-1, df_group.columns.get_loc('Группировка')] = 'Все страны'
  row_all = df_group.loc[df_group.index[-1]].to_frame().T
  df_group.drop(df_group.tail(1).index, inplace=True)
  df_group = pd.concat([df_group, row_all], axis=0).sort_values(by=2022, ascending=False).reset_index(drop=True)

  return df_group

def calculate_structure_table(df):
  df_structure = df.loc[:,2001:].div(df.loc[0,2001:])*100
  df_structure_name = pd.concat([df['Группировка'], df_structure], axis=1).reset_index(drop=True)
  return df_structure_name

In [ ]:
def calclulate_percent(df):
  df_change = df.select_dtypes(include=['int','float']).pct_change(axis=1).fillna(0)
  #df_change[2001].replace(0, 1, inplace=True)
  df_change += 1
  df_change_cumproded = df_change.cumprod(axis=1)
  df_change_cumproded.insert(loc=0, column='Группировка', value=df['Группировка'])
  return df_change_cumproded

In [ ]:
# CA = pd.read_excel(all_files_path[0])
# df_original, df_money, df_structure = get_main_tables(CA)
# for col in df_money.columns:
#   df_money[col] = pd.to_numeric(df_money[col], errors='ignore')

# calclulate_percent(df_money)

## Расчеты

In [ ]:
# CA = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Импорт_ЦА_страны.xlsx')
# CA.head()

In [ ]:
def get_main_tables(df):

  df_with_years = change_colnames(df, cols = col_names_countries)
  df_with_countries_groups = merge_columns(df_with_years)
  df_grouped_by_groups = sum_by_countriesgroup(df_with_countries_groups)
  df_group_structure = calculate_structure_table(df_grouped_by_groups)
  return df_with_countries_groups, df_grouped_by_groups, df_group_structure

In [ ]:
def make_dict_of_dfs(df_original, df_money, df_structure, df_change):
  my_dfs = {}
  my_dfs['Первичные'] = df_original
  my_dfs['Сумма'] = df_money
  my_dfs['Структура'] = df_structure
  my_dfs['Процент_коммулятивно'] = df_change
  return my_dfs

In [ ]:
def convert_cols_float(df):
  for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')
  return df

In [ ]:
def write_to_excel(my_dfs, file_name):
  filename = f'/content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_{file_name}.xlsx'
  writer = pd.ExcelWriter(filename, engine='xlsxwriter')
  for sheetname, df in my_dfs.items():  # loop through `dict` of dataframes
      df.to_excel(writer, sheet_name=sheetname, index = False)  # send df to writer
      worksheet = writer.sheets[sheetname]  # pull worksheet object
      for idx, col in enumerate(df):  # loop through all columns
          series = df[col]
          max_len = max((
              series.astype(str).map(len).max(),  # len of largest item
              len(str(series.name))  # len of column name/header
              )) + 1  # adding a little extra space
          worksheet.set_column(idx, idx, max_len)  # set column width
  writer.close()


In [ ]:
# 'товары' in all_files_path[2]

In [ ]:
# df = pd.read_excel(all_files_path[0])
# df_original, df_money, df_structure = get_main_tables(df)
# df_money

## Узбекистан

In [ ]:
Uzb_import_till_2016 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Узб_импорт_до_2016_страны.xlsx')
Uzb_import_till_2016.head()

,Экспортеры,стоимость Импорта в 2001,стоимость Импорта в 2002,стоимость Импорта в 2003,стоимость Импорта в 2004,стоимость Импорта в 2005,стоимость Импорта в 2006,стоимость Импорта в 2007,стоимость Импорта в 2008,стоимость Импорта в 2009,стоимость Импорта в 2010,стоимость Импорта в 2011,стоимость Импорта в 2012,стоимость Импорта в 2013,стоимость Импорта в 2014,стоимость Импорта в 2015,стоимость Импорта в 2016
0,Китай,50684.00,104374.00,146783.00,172442.00,230064.00,406154.00,764854.00,1277814.00,1560539.00,1181024.00,1359242.00,1783339.00,2613355.00,2678214.00,2236357.00,2033302.00
1,Казахстан,150234.00,101022.00,137802.00,201674.00,242564.00,384583.00,871733.00,1272045.00,891846.00,1101225.00,1179464.00,1343721.00,1145310.00,1083947.00,942267.00,922532.00
2,Республика Корея,345687.00,188479.00,247138.00,359048.00,493095.00,649302.00,748302.00,1122592.00,1149783.00,1438644.00,1718758.00,1766516.00,1968246.00,2032719.00,1284247.00,930468.00
3,Турция,89725.00,93735.00,138422.00,145226.00,151071.00,175995.00,225612.00,337130.00,279964.00,282666.00,354490.00,449884.00,624482.00,650355.00,488652.00,533322.00
4,Германия,201709.00,201711.00,215817.00,220829.00,284926.00,308533.00,360441.00,465050.00,472348.00,721186.00,685861.00,517262.00,572294.00,671104.00,465138.00,480078.00


In [ ]:
Uzb_import_till_2016.shape

(138, 17)

In [ ]:
Uzb_import_till_2022 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Узбекистан_импорт_2017_2022.xlsx')
Uzb_import_till_2022

,Экспортеры,стоимость Импорта в 2017,стоимость Импорта в 2018,стоимость Импорта в 2019,стоимость Импорта в 2020,стоимость Импорта в 2021,стоимость Импорта в 2022
0,Китай,2700379,3539417,5052269,4425626,4860715.00,6321465.00
1,Российская Федерация,2563885,3382752,3974225,4079758,5331691.00,6053274.00
2,Казахстан,975267,1509389,1878388,2091366,2701119.00,3222244.00
3,Республика Корея,1156709,1936163,2524435,1934612,1742089.00,2171776.00
4,Турция,594443,1096622,1296689,1072526,1655968.00,1674200.00
...,...,...,...,...,...,...,...
160,Парагвай,0,0,0,22,30.00,NaN
161,Сенегал,0,0,1,0,3.00,NaN
162,Сейшельские Острова,0,0,0,1,1.00,NaN
163,Суринам,0,0,0,33,0.00,NaN


In [ ]:
Uzb_import = Uzb_import_till_2016.merge(Uzb_import_till_2022, how='outer', on='Экспортеры').fillna(0)
Uzb_import

,Экспортеры,стоимость Импорта в 2001,стоимость Импорта в 2002,стоимость Импорта в 2003,стоимость Импорта в 2004,стоимость Импорта в 2005,стоимость Импорта в 2006,стоимость Импорта в 2007,стоимость Импорта в 2008,стоимость Импорта в 2009,стоимость Импорта в 2010,стоимость Импорта в 2011,стоимость Импорта в 2012,стоимость Импорта в 2013,стоимость Импорта в 2014,стоимость Импорта в 2015,стоимость Импорта в 2016,стоимость Импорта в 2017,стоимость Импорта в 2018,стоимость Импорта в 2019,стоимость Импорта в 2020,стоимость Импорта в 2021,стоимость Импорта в 2022
0,Китай,50684.00,104374.00,146783.00,172442.00,230064.00,406154.00,764854.00,1277814.00,1560539.00,1181024.00,1359242.00,1783339.00,2613355.00,2678214.00,2236357.00,2033302.00,2700379.00,3539417.00,5052269.00,4425626.00,4860715.00,6321465.00
1,Казахстан,150234.00,101022.00,137802.00,201674.00,242564.00,384583.00,871733.00,1272045.00,891846.00,1101225.00,1179464.00,1343721.00,1145310.00,1083947.00,942267.00,922532.00,975267.00,1509389.00,1878388.00,2091366.00,2701119.00,3222244.00
2,Республика Корея,345687.00,188479.00,247138.00,359048.00,493095.00,649302.00,748302.00,1122592.00,1149783.00,1438644.00,1718758.00,1766516.00,1968246.00,2032719.00,1284247.00,930468.00,1156709.00,1936163.00,2524435.00,1934612.00,1742089.00,2171776.00
3,Турция,89725.00,93735.00,138422.00,145226.00,151071.00,175995.00,225612.00,337130.00,279964.00,282666.00,354490.00,449884.00,624482.00,650355.00,488652.00,533322.00,594443.00,1096622.00,1296689.00,1072526.00,1655968.00,1674200.00
4,Германия,201709.00,201711.00,215817.00,220829.00,284926.00,308533.00,360441.00,465050.00,472348.00,721186.00,685861.00,517262.00,572294.00,671104.00,465138.00,480078.00,574056.00,699749.00,884310.00,691915.00,645973.00,1010473.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Бенин,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36.00,0.00,1.00,0.00
170,Гренада,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00
171,Мальдивские Острова,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,110.00,64.00,0.00,0.00
172,Новая Каледония,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00


In [ ]:
Uzb_import.to_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Импорт_узбекистан_страны.xlsx')

In [ ]:
Uzb_export_till_2016 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Узбекистан_экспорт_до_2016_страны.xlsx')
Uzb_export_till_2016.head()

,Импортеры,стоимость Экспорта в 2001,стоимость Экспорта в 2002,стоимость Экспорта в 2003,стоимость Экспорта в 2004,стоимость Экспорта в 2005,стоимость Экспорта в 2006,стоимость Экспорта в 2007,стоимость Экспорта в 2008,стоимость Экспорта в 2009,стоимость Экспорта в 2010,стоимость Экспорта в 2011,стоимость Экспорта в 2012,стоимость Экспорта в 2013,стоимость Экспорта в 2014,стоимость Экспорта в 2015,стоимость Экспорта в 2016
0,Швейцария,630539.00,304819.00,592947.00,680615.00,1196653.00,610985.00,1401049.00,3381460.00,2710656.00,32843.00,9375.00,543185.00,1539042.00,1337660.00,1872861.00,2844489.00
1,Китай,7617.00,27403.00,200251.00,403066.00,450495.00,565937.00,363450.00,328882.00,349419.00,1300780.00,807315.00,1091833.00,1938093.00,1597910.00,1266792.00,1606926.00
2,Турция,36045.00,75342.00,99462.00,178671.00,261466.00,415841.00,613810.00,580810.00,413079.00,861373.00,939882.00,813287.00,853058.00,846432.00,711555.00,709292.00
3,Казахстан,81085.00,86523.00,89714.00,227608.00,254456.00,319090.00,538470.00,516081.00,304414.00,473311.00,770558.00,817200.00,970083.00,1017662.00,725442.00,587792.00
4,Соединенное Королевство Великобритании и Север...,259791.00,816157.00,535183.00,693127.00,188662.00,168415.00,74147.00,785534.00,52038.00,103621.00,54572.00,42188.00,12324.00,1586.00,2681.00,2887.00


In [ ]:
Uzb_export_till_2022 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Узбекистан_экспорт_2017_2022_страны.xlsx')
Uzb_export_till_2022.head()

,Импортеры,стоимость Экспорта в 2017,стоимость Экспорта в 2018,стоимость Экспорта в 2019,стоимость Экспорта в 2020,стоимость Экспорта в 2021,стоимость Экспорта в 2022
0,"Территории, более нигде не включенные",3645206,3237880,5629420,6465204,4802820.00,4575512.00
1,Российская Федерация,1453672,1636681,2067207,1161766,1703536.00,2599858.00
2,Китай,1313428,2120927,1767642,1282105,1744424.00,1753107.00
3,Турция,833514,865828,1149658,949066,1638927.00,1457562.00
4,Казахстан,977256,1219611,1237315,732315,1023478.00,1251475.00


In [ ]:
Uzb_export = Uzb_export_till_2016.merge(Uzb_export_till_2022, how='outer', on='Импортеры').fillna(0)
Uzb_export

,Импортеры,стоимость Экспорта в 2001,стоимость Экспорта в 2002,стоимость Экспорта в 2003,стоимость Экспорта в 2004,стоимость Экспорта в 2005,стоимость Экспорта в 2006,стоимость Экспорта в 2007,стоимость Экспорта в 2008,стоимость Экспорта в 2009,стоимость Экспорта в 2010,стоимость Экспорта в 2011,стоимость Экспорта в 2012,стоимость Экспорта в 2013,стоимость Экспорта в 2014,стоимость Экспорта в 2015,стоимость Экспорта в 2016,стоимость Экспорта в 2017,стоимость Экспорта в 2018,стоимость Экспорта в 2019,стоимость Экспорта в 2020,стоимость Экспорта в 2021,стоимость Экспорта в 2022
0,Швейцария,630539.00,304819.00,592947.00,680615.00,1196653.00,610985.00,1401049.00,3381460.00,2710656.00,32843.00,9375.00,543185.00,1539042.00,1337660.00,1872861.00,2844489.00,3563.00,3221.00,21401.00,5795.00,17373.00,12370.00
1,Китай,7617.00,27403.00,200251.00,403066.00,450495.00,565937.00,363450.00,328882.00,349419.00,1300780.00,807315.00,1091833.00,1938093.00,1597910.00,1266792.00,1606926.00,1313428.00,2120927.00,1767642.00,1282105.00,1744424.00,1753107.00
2,Турция,36045.00,75342.00,99462.00,178671.00,261466.00,415841.00,613810.00,580810.00,413079.00,861373.00,939882.00,813287.00,853058.00,846432.00,711555.00,709292.00,833514.00,865828.00,1149658.00,949066.00,1638927.00,1457562.00
3,Казахстан,81085.00,86523.00,89714.00,227608.00,254456.00,319090.00,538470.00,516081.00,304414.00,473311.00,770558.00,817200.00,970083.00,1017662.00,725442.00,587792.00,977256.00,1219611.00,1237315.00,732315.00,1023478.00,1251475.00
4,Соединенное Королевство Великобритании и Север...,259791.00,816157.00,535183.00,693127.00,188662.00,168415.00,74147.00,785534.00,52038.00,103621.00,54572.00,42188.00,12324.00,1586.00,2681.00,2887.00,21714.00,134265.00,93883.00,16971.00,11220.00,23019.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,Ангилья,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.00,0.00,0.00,0.00,0.00,0.00
173,Внешние малые острова Соединенных Штатов,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,16.00,0.00
174,Центральноафриканская Республика,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,26.00,0.00
175,Доминика,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,910.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
Uzb_export.to_excel('/content/drive/MyDrive/Trade_CA/ЦА/Узбекистан/Экспорт_узбекистан_страны.xlsx')

## Таджикистан

In [ ]:
Taj_import2013 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Импорт_Таджикистан_страны_до2013.xlsx')
Taj_import2013 = Taj_import2013.fillna(0)
Taj_import2013.head()

,Экспортеры,стоимость Импорта в 2001,стоимость Импорта в 2002,стоимость Импорта в 2003,стоимость Импорта в 2004,стоимость Импорта в 2005,стоимость Импорта в 2006,стоимость Импорта в 2007,стоимость Импорта в 2008,стоимость Импорта в 2009,стоимость Импорта в 2010,стоимость Импорта в 2011,стоимость Импорта в 2012,стоимость Импорта в 2013
0,Китай,5301.00,6501.00,20808.00,53561.00,143738.00,305778.00,513766.00,1479683.00,1217574.00,1376501.00,1996778.00,1747870.00,1869364.00
1,Казахстан,61198.00,45740.00,75736.00,136126.00,150609.00,157351.00,278466.00,268510.00,240347.00,259737.00,356778.00,535327.00,496839.00
2,Узбекистан,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Турция,15553.00,10915.00,29478.00,41637.00,46717.00,71787.00,118107.00,176448.00,126364.00,143890.00,172575.00,234947.00,309023.00
4,Объединенные Арабские Эмираты,0.00,0.00,0.00,0.00,29966.00,0.00,39549.00,24792.00,0.00,0.00,0.00,72535.00,50603.00


In [ ]:
Taj_import2022 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Импорт_Таджикистан_страны_до2022.xlsx')
Taj_import2022 = Taj_import2022.fillna(0)
Taj_import2022.head()

,Экспортеры,стоимость Импорта в 2014,стоимость Импорта в 2015,стоимость Импорта в 2016,стоимость Импорта в 2017,стоимость Импорта в 2018,стоимость Импорта в 2019,стоимость Импорта в 2020,стоимость Импорта в 2021,стоимость Импорта в 2022
0,Российская Федерация,1353342,1058618,985864,903855,967937,1009128,932593,1280469.00,1585687.00
1,Казахстан,617096,530893,481082,514639,536594,739335,757571,818533.00,957266.00
2,Китай,713694,764513,841305,556182,594154,605468,438392,679797.00,841707.00
3,Узбекистан,5086,8831,33630,69334,126263,169949,238447,326076.00,392460.00
4,Турция,146139,127726,114545,105368,124915,143477,119617,158275.00,171644.00


In [ ]:
Taj_import = Taj_import2013.merge(Taj_import2022, how='outer', on='Экспортеры').fillna(0)
Taj_import

,Экспортеры,стоимость Импорта в 2001,стоимость Импорта в 2002,стоимость Импорта в 2003,стоимость Импорта в 2004,стоимость Импорта в 2005,стоимость Импорта в 2006,стоимость Импорта в 2007,стоимость Импорта в 2008,стоимость Импорта в 2009,стоимость Импорта в 2010,стоимость Импорта в 2011,стоимость Импорта в 2012,стоимость Импорта в 2013,стоимость Импорта в 2014,стоимость Импорта в 2015,стоимость Импорта в 2016,стоимость Импорта в 2017,стоимость Импорта в 2018,стоимость Импорта в 2019,стоимость Импорта в 2020,стоимость Импорта в 2021,стоимость Импорта в 2022
0,Китай,5301.00,6501.00,20808.00,53561.00,143738.00,305778.00,513766.00,1479683.00,1217574.00,1376501.00,1996778.00,1747870.00,1869364.00,713694.00,764513.00,841305.00,556182.00,594154.00,605468.00,438392.00,679797.00,841707.00
1,Казахстан,61198.00,45740.00,75736.00,136126.00,150609.00,157351.00,278466.00,268510.00,240347.00,259737.00,356778.00,535327.00,496839.00,617096.00,530893.00,481082.00,514639.00,536594.00,739335.00,757571.00,818533.00,957266.00
2,Узбекистан,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5086.00,8831.00,33630.00,69334.00,126263.00,169949.00,238447.00,326076.00,392460.00
3,Турция,15553.00,10915.00,29478.00,41637.00,46717.00,71787.00,118107.00,176448.00,126364.00,143890.00,172575.00,234947.00,309023.00,146139.00,127726.00,114545.00,105368.00,124915.00,143477.00,119617.00,158275.00,171644.00
4,Объединенные Арабские Эмираты,0.00,0.00,0.00,0.00,29966.00,0.00,39549.00,24792.00,0.00,0.00,0.00,72535.00,50603.00,22733.00,18549.00,21511.00,23307.00,23159.00,21666.00,14842.00,19706.00,20904.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Маврикий,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00,0.00
149,Кюрасао,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,33.00,0.00
150,Сент-Винсент и Гренадины,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,39.00,0.00
151,Сейшельские Острова,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,14.00,0.00,53.00,0.00,0.00,11.00,0.00


In [ ]:
Taj_import.to_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Импорт_Таджикистан_всестраны.xlsx')

In [ ]:
Taj_export2013 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Экспорт_таджикистан_до2013.xlsx')
Taj_export2013= Taj_export2013.fillna(0)
Taj_export2013.head()

,Импортеры,стоимость Экспорта в 2001,стоимость Экспорта в 2002,стоимость Экспорта в 2003,стоимость Экспорта в 2004,стоимость Экспорта в 2005,стоимость Экспорта в 2006,стоимость Экспорта в 2007,стоимость Экспорта в 2008,стоимость Экспорта в 2009,стоимость Экспорта в 2010,стоимость Экспорта в 2011,стоимость Экспорта в 2012,стоимость Экспорта в 2013
0,Казахстан,2254.00,2993.00,7255.00,3478.00,16951.00,27714.00,25006.00,8646.00,14792.00,16615.00,47810.00,68241.00,72637.00
1,Китай,5452.00,5886.00,18008.00,15366.00,14197.00,18002.00,10285.00,20244.00,184985.00,56048.00,72228.00,108829.00,88751.00
2,Швейцария,1779.00,91.00,6663.00,16928.00,14307.00,11579.00,12436.00,14622.00,7467.00,182.00,21.00,53734.00,50313.00
3,Турция,13662.00,40696.00,56999.00,62818.00,47309.00,118395.00,143352.00,147727.00,107267.00,283689.00,324283.00,345178.00,441750.00
4,Узбекистан,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
Taj_export2022 = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Экспорт_таджикистан_до2022.xlsx')
Taj_export2022 = Taj_export2022.fillna(0)
Taj_export2022.head()

,Импортеры,стоимость Экспорта в 2014,стоимость Экспорта в 2015,стоимость Экспорта в 2016,стоимость Экспорта в 2017,стоимость Экспорта в 2018,стоимость Экспорта в 2019,стоимость Экспорта в 2020,стоимость Экспорта в 2021,стоимость Экспорта в 2022
0,Весь Мир,813493,897605,899320,983826,1073858,1115850,1311879,1786916.00,2330041.00
1,Швейцария,82319,166972,99309,1490,1640,225127,595115,534977.00,574556.00
2,Казахстан,177619,146452,194860,324338,299974,218379,152214,360311.00,483915.00
3,Китай,39149,29512,44045,35381,57076,55419,34411,159619.00,368335.00
4,Турция,254952,255264,200765,233227,276524,233798,203032,232168.00,179554.00


In [ ]:
Taj_export = Taj_export2013.merge(Taj_export2022, how='outer', on='Импортеры').fillna(0)
Taj_export

,Импортеры,стоимость Экспорта в 2001,стоимость Экспорта в 2002,стоимость Экспорта в 2003,стоимость Экспорта в 2004,стоимость Экспорта в 2005,стоимость Экспорта в 2006,стоимость Экспорта в 2007,стоимость Экспорта в 2008,стоимость Экспорта в 2009,стоимость Экспорта в 2010,стоимость Экспорта в 2011,стоимость Экспорта в 2012,стоимость Экспорта в 2013,стоимость Экспорта в 2014,стоимость Экспорта в 2015,стоимость Экспорта в 2016,стоимость Экспорта в 2017,стоимость Экспорта в 2018,стоимость Экспорта в 2019,стоимость Экспорта в 2020,стоимость Экспорта в 2021,стоимость Экспорта в 2022
0,Казахстан,2254.00,2993.00,7255.00,3478.00,16951.00,27714.00,25006.00,8646.00,14792.00,16615.00,47810.00,68241.00,72637.00,177619.00,146452.00,194860.00,324338.00,299974.00,218379.00,152214.00,360311.00,483915.00
1,Китай,5452.00,5886.00,18008.00,15366.00,14197.00,18002.00,10285.00,20244.00,184985.00,56048.00,72228.00,108829.00,88751.00,39149.00,29512.00,44045.00,35381.00,57076.00,55419.00,34411.00,159619.00,368335.00
2,Швейцария,1779.00,91.00,6663.00,16928.00,14307.00,11579.00,12436.00,14622.00,7467.00,182.00,21.00,53734.00,50313.00,82319.00,166972.00,99309.00,1490.00,1640.00,225127.00,595115.00,534977.00,574556.00
3,Турция,13662.00,40696.00,56999.00,62818.00,47309.00,118395.00,143352.00,147727.00,107267.00,283689.00,324283.00,345178.00,441750.00,254952.00,255264.00,200765.00,233227.00,276524.00,233798.00,203032.00,232168.00,179554.00
4,Узбекистан,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3308.00,6106.00,35640.00,57081.00,155269.00,171975.00,82372.00,126670.00,179549.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Британские Виргинские острова,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2048.00,3652.00,4934.00,6811.00,6723.00,12969.00,16204.00
153,Туркменистан,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,982.00,2445.00,1423.00,1508.00,589.00,260.00,0.00,6.00,783.00
154,Ирак,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1729.00,1567.00,3839.00,3513.00,1062.00,710.00,733.00,383.00,350.00
155,Экваториальная Гвинея,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00


In [ ]:
Taj_export.to_excel('/content/drive/MyDrive/Trade_CA/ЦА/Таджикистан/Экспорт_Таджикистан_всестраны.xlsx')

## Импорт файлов

In [ ]:
all_files_path = []
all_files_names= []
i = 0

dirpath = pathlib.Path('/content/drive/MyDrive/Trade_CA/ЦА/')

for path in dirpath.glob("*.xlsx"):
  all_files_path.append(str(path))
  all_files_names.append(all_files_path[i].split('/')[-1].split('.')[0])

print(all_files_path)
print(all_files_names)

['/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_ЦА_страны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Импорт_ЦА_страны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_узбекистан_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Импорт_узбекистан_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Импорт_Казахстан_все_страны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_Казахстан_все_страны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_Кыргызстан_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Ипорт_Кыргызстан_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Импорт_Туркмения_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_Туркмения_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Импорт_Таджикистан_всестраны.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Экспорт_Таджикистан_всестраны.xlsx']
['Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_страны', 'Экспорт_ЦА_стра

In [ ]:
# CA = pd.read_excel(all_files_path[0])
# df_original, df_money, df_structure = get_main_tables(CA)
# for col in df_money.columns:
#   df_money[col] = pd.to_numeric(df_money[col], errors='ignore')


In [ ]:
for file in all_files_path:
  file_name = file.split('/')[-1].split('.')[0]
  df = pd.read_excel(file)
  df_original, df_money, df_structure = get_main_tables(df)
  df_money = convert_cols_float(df_money)
  df_change = calclulate_percent(df_money)
  # print(df_change)

  my_df = make_dict_of_dfs(df_original, df_money, df_structure, df_change)
  write_to_excel(my_df, file_name)

In [ ]:
!zip -r /content/drive/MyDrive/Trade_CA/ЦА/Анализ/all_countries.zip /content/drive/MyDrive/Trade_CA/ЦА/Анализ

  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/ (stored 0%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Экспорт_ЦА_страны.xlsx (deflated 7%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Импорт_ЦА_страны.xlsx (deflated 4%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Импорт_узбекистан_всестраны.xlsx (deflated 8%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Экспорт_узбекистан_всестраны.xlsx (deflated 8%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Импорт_Казахстан_все_страны.xlsx (deflated 5%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Экспорт_Казахстан_все_страны.xlsx (deflated 6%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Экспорт_Кыргызстан_всестраны.xlsx (deflated 8%)
  adding: content/drive/MyDrive/Trade_CA/ЦА/Анализ/Анализ_Ипорт_Кыргызстан_всестраны.xlsx (deflated 7%)
  adding: content/drive/MyDrive/

## Товары

In [ ]:
# df_goods = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/0-full_cllassification_final.xlsx')
# df_goods.head()

In [ ]:
# df_fullgoods = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/Классификатор_полный.xlsx')
# df_fullgoods.head()

In [ ]:
# df_goods.loc[df_goods['Код'].str.contains('0101')]

In [ ]:
# full_classification = df_fullgoods.merge(df_goods, how='left', on = 'Код')
# full_classification = full_classification.drop(columns={'Наименование товара_y'})
# full_classification.rename(columns={'Наименование товара_x':'Наименование товара'}, inplace= True)
# full_classification = full_classification.fillna(0)
# full_classification = full_classification.sort_values(by = 'Код').reset_index(drop=True)
# full_classification

In [ ]:
# full_classification.loc[full_classification['Отрасль']==0]

In [ ]:
# full_classification.to_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/full_cllassification.xlsx')

In [ ]:
# df_export = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/ЦА/Экспорт_ЦА_товары.xlsx')
# df_export.head()

In [ ]:
def remove_symbol(s):
  return s.replace("'","")

In [ ]:
# df_goods['Код'] = df_goods['Код'].apply(remove_symbol)
# df_goods

In [ ]:
# df_export['Код'] = df_export['Код'].apply(remove_symbol)
# df_import['Код'] = df_import['Код'].apply(remove_symbol)
# df_kz_export['Код'] = df_kz_export['Код'].apply(remove_symbol)

In [ ]:
ind_order = {'Все товары':0, 'Горючие полезные ископаемые и первичная продукция из них':1, 'Уран, радиоактивные вещества и прочие изотопы':2, 'Электроэнергия':3,
             'Нерудные полезные ископаемые и первичная продукция из них':4, 'Рудные полезные ископаемые и первичная продукция из них':5, 'Драгоценные и полудрагоценыне камни и металлы и изделия из них':6,
             'Сырье для легкой промышленности (шкуры, кожа, мех, пряжа, ткани и пр.)':7, 'Продукция легкой промышленности':8, 'Изделия из древисины, включая бумажную продукцию':9,
             'Изделия из нерудных материалов (камень, кермика, стекло и пр.)':10, 'Продукция из пластмассы и резины, включая сырье':11, 'Фармацевтическая продукция':12,
             'Продукция химической промышленности':13, 'Изделия из металлов':14, 'Машины и оборудование (кроме транспортных)':15, 'Транспортные средства и оборудование':16,
             'Прочая промышленная непродовольственная продукция':17, 'Мясо-молочная и рыбная продукция, включая сырье':18, 'Зерновые и бобовые, продукция из них':19,
             'Плодоовощная продукция, включая сырье':20, 'Прочие продовольственыне товары и сырье для них':21, 'Прочее':22, 'Товары нигде не указанные':23}

In [ ]:
def change_colnames(df):
  num_of_years = len(df._get_numeric_data().columns)
  begin_year = 2023- num_of_years
  col_names_goods = list(range(begin_year,2023))
  col_names_goods.insert(0,'Код')
  col_names_goods.insert(1,'Наименование товара')
  df = df.fillna(0)
  df.columns = col_names_goods
  return df

In [ ]:
def merge_columns_groups (df, good_groups, ind_order):
  good_groups = good_groups.drop('Наименование товара', axis=1, inplace = True)
  df_grouped = df.merge(df_goods, how = 'left', on = 'Код').fillna(0)
  grouped_column = df_grouped.pop('Отрасль')
  df_grouped.insert(loc=1, column='Отрасль', value=grouped_column)
  df_grouped = df_grouped.sort_values(by='Код', ascending = True)
  return df_grouped

In [ ]:
# df_merged.loc[df_merged['Отрасль']==0]

In [ ]:
# print(df_merged.shape)
# print(df_goods.shape)

In [ ]:
def sum_by_industrygroup(df, ind_order):
  df_group = df.melt(id_vars=["Код", "Отрасль", "Наименование товара"], var_name="Year", value_name="Value")
  #df_group = df_group.replace(0, 1)
  df_group = df_group.pivot_table(values = 'Value', index = 'Отрасль', columns= 'Year', aggfunc='sum').reset_index()
  df_group.loc['Total']= df_group.sum(numeric_only=True)
  df_group = df_group.reset_index(drop=True)
  df_group.iloc[-1, df_group.columns.get_loc('Отрасль')] = 'Все товары'
  row_all = df_group.loc[df_group.index[-1]].to_frame().T
  df_group.drop(df_group.tail(1).index, inplace=True)
  df_group = pd.concat([df_group, row_all], axis=0).sort_values(by='Отрасль',  key=lambda x: x.map(ind_order)).reset_index(drop=True)

  return df_group


In [ ]:
def convert_cols_float(df):
  for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')
  return df

# df_industry_by_years = convert_cols_float(df_industry_by_years)
# df_industry_by_years

In [ ]:
def calculate_structure_table(df):
  num_of_years = len(df._get_numeric_data().columns)
  begin_year = 2023- num_of_years
  print(begin_year)
  df_structure = df.loc[:,begin_year:].div(df.loc[0,begin_year:])*100
  df_structure_name = pd.concat([df['Отрасль'], df_structure], axis=1).reset_index(drop=True)
  return df_structure_name

# calculate_structure_table(df_industry_by_years)

In [ ]:
def all_actions_goods(df, good_groups, ind_order):
  df_cols = change_colnames(df)
  df_merged = merge_columns_groups (df_cols, good_groups, ind_order)
  df_sum_by_industry = sum_by_industrygroup(df_merged, ind_order)
  df_sum_by_industry = convert_cols_float(df_sum_by_industry)
  df_structure = calculate_structure_table(df_sum_by_industry)
  return df_merged, df_sum_by_industry, df_structure

In [ ]:
def make_dict_of_dfs(df_merged, df_sum_by_industry, df_structure):
  my_dfs = {}
  my_dfs['Первичные'] = df_merged
  my_dfs['Отрасли'] = df_sum_by_industry
  my_dfs['Структура'] = df_structure
  return my_dfs

In [ ]:
def write_to_excel(my_dfs, file_name):
  filename = f'/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_{file_name}.xlsx'
  writer = pd.ExcelWriter(filename, engine='xlsxwriter')
  for sheetname, df in my_dfs.items():  # loop through `dict` of dataframes
      df.to_excel(writer, sheet_name=sheetname, index = False)  # send df to writer
      worksheet = writer.sheets[sheetname]  # pull worksheet object
      for idx, col in enumerate(df):  # loop through all columns
          series = df[col]
          max_len = max((
              series.astype(str).map(len).max(),  # len of largest item
              len(str(series.name))  # len of column name/header
              )) + 1  # adding a little extra space
          worksheet.set_column(idx, idx, max_len)  # set column width
  writer.close()

In [ ]:
all_files_path = []
all_files_names= []
i = 0

dirpath = pathlib.Path('/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/')

for path in dirpath.glob("*.xlsx"):
  all_files_path.append(str(path))
  all_files_names.append(all_files_path[i].split('/')[-1].split('.')[0])

print(all_files_path)
print(all_files_names)

['/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Казахстан_экспорт_в_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Казахстан_импорт_из_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Таджикистан_импорт_из_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Таджикистан_экспорт_в_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Узбекистан_импорт_из_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Кыргызстан_импорт_из_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Туркменистан_импорт_из_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Туркменистан_экспорт_в_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Кыргызстан_экспорт_в_ЦА.xlsx', '/content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Узбекистан_экспорт_в_ЦА.xlsx']
['Казахстан_экспорт_в_ЦА', 'Казахстан_экспорт_в_ЦА', 'Казахстан_экспорт_в_ЦА', 'Казахстан_экспо

In [ ]:
for file in all_files_path:
  df_goods = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/0-full_cllassification_final.xlsx')
  #df_goods['Код'] = df_goods['Код'].apply(remove_symbol)
  file_name = file.split('/')[-1].split('.')[0]
  print(file_name)
  df = pd.read_excel(file)
  df_merged, df_sum_by_industry, df_structure  = all_actions_goods(df, df_goods, ind_order)
  my_df = make_dict_of_dfs(df_merged, df_sum_by_industry, df_structure)
  write_to_excel(my_df, file_name)

Казахстан_экспорт_в_ЦА
2003
Казахстан_импорт_из_ЦА
2003
Таджикистан_импорт_из_ЦА
2014
Таджикистан_экспорт_в_ЦА
2014
Узбекистан_импорт_из_ЦА
2017
Кыргызстан_импорт_из_ЦА
2003
Туркменистан_импорт_из_ЦА
2017
Туркменистан_экспорт_в_ЦА
2017
Кыргызстан_экспорт_в_ЦА
2003
Узбекистан_экспорт_в_ЦА
2017


In [ ]:
!zip -r /content/all_goods_CA.zip /content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/

updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/ (stored 0%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Казахстан_экспорт_в_ЦА.xlsx (deflated 10%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Казахстан_импорт_из_ЦА.xlsx (deflated 14%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Таджикистан_импорт_из_ЦА.xlsx (deflated 10%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Таджикистан_экспорт_в_ЦА.xlsx (deflated 18%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Узбекистан_импорт_из_ЦА.xlsx (deflated 6%)
updating: content/drive/MyDrive/Trade_CA/ЦА/Товары/Trade_inside_CA/Trade_inside_CA_Analysys/Анализ_Кыргызстан_импорт_из_ЦА.xlsx (deflated 12%)
updating: content/drive/MyDrive/Trade_

## Товары / страны

In [ ]:
df_goods = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Товары/0-full_cllassification_final.xlsx')
df_goods.head()

,Код,Наименование товара,Отрасль
0,'0101,"Лошади, ослы, мулы и лошаки живые","Мясо-молочная и рыбная продукция, включая сырье"
1,'0102,Крупный рогатый скот живой,"Мясо-молочная и рыбная продукция, включая сырье"
2,'0103,Свиньи живые,"Мясо-молочная и рыбная продукция, включая сырье"
3,'0104,Овцы и козы живые,"Мясо-молочная и рыбная продукция, включая сырье"
4,'0105,"Домашняя птица живая, то есть куры домашние (g...","Мясо-молочная и рыбная продукция, включая сырье"


In [ ]:
df_countries = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Страны_товары/ЦА_товар_страна.xlsx')
df_countries.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'2709,"Нефть сырая и нефтепродукты сырые, полученные ...",26584364.00,37796201.00,33563064.00,23703746.00,31089836.00,46919552.00,Экспорт,Все страны,Казахстан
1,'7403,Медь рафинированная и сплавы медные необработа...,2342492.00,2429709.00,2504802.00,2720389.00,3260869.00,3746968.00,Экспорт,Все страны,Казахстан
2,'7202,Ферросплавы,2205703.00,2203421.00,1883723.00,1657952.00,2279980.00,3230575.00,Экспорт,Все страны,Казахстан
3,'2844,Элементы химические радиоактивные и изотопы ра...,1442061.00,1345946.00,1549235.00,1718381.00,1764910.00,2641658.00,Экспорт,Все страны,Казахстан
4,'2603,Руды и концентраты медные,1080350.00,1185482.00,1153834.00,1462860.00,1606797.00,2366252.00,Экспорт,Все страны,Казахстан


In [ ]:
df_countries.shape

(83671, 11)

In [ ]:
df_countries2 = df_countries.loc[df_countries['Группа стран']!='Все страны'].reset_index(drop=True)
df_countries2.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'1001,Пшеница и меслин,402577.00,504833.00,645482.00,857613.00,941508.00,1259182.00,Экспорт,ЦА,Казахстан
1,'8517,"Аппараты телефонные, включая аппараты телефонн...",3440.00,3172.00,5455.00,5848.00,336001.00,496208.00,Экспорт,ЦА,Казахстан
2,'2603,Руды и концентраты медные,10640.00,4069.00,87018.00,1984.00,53655.00,259178.00,Экспорт,ЦА,Казахстан
3,'8703,Автомобили легковые и прочие моторные транспор...,15486.00,14546.00,24444.00,109533.00,137512.00,251614.00,Экспорт,ЦА,Казахстан
4,'7207,Полуфабрикаты из железа или нелегированной стали,138150.00,168075.00,158217.00,122224.00,199007.00,223149.00,Экспорт,ЦА,Казахстан


In [ ]:
cols = df_countries2.columns.to_list()
cols

['Код',
 'Продукты',
 'Стоимость в 2017',
 'Стоимость в 2018',
 'Стоимость в 2019',
 'Стоимость в 2020',
 'Стоимость в 2021',
 'Стоимость в 2022',
 'Направление',
 'Группа стран',
 'Страна']

In [ ]:
df_countries2.shape

(72002, 11)

In [ ]:
df_all_countries = pd.read_excel('/content/drive/MyDrive/Trade_CA/ЦА/Страны_товары/ЦА_Товар_страна_все_страны_прямые.xlsx')
df_all_countries.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан
1,'5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан
2,'2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан
3,'7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан
4,'9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан


In [ ]:
df_all_countries.columns = cols
df_all_countries.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан
1,'5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан
2,'2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан
3,'7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан
4,'9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан


In [ ]:
df_countries3 = pd.concat([df_all_countries, df_countries2], axis=0)
df_countries3.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан
1,'5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан
2,'2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан
3,'7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан
4,'9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан


In [ ]:
df_countries = df_countries3.copy()
df_countries.shape

(82904, 11)

In [ ]:
df_countries.head()

,Код,Продукты,Стоимость в 2017,Стоимость в 2018,Стоимость в 2019,Стоимость в 2020,Стоимость в 2021,Стоимость в 2022,Направление,Группа стран,Страна
0,'7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан
1,'5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан
2,'2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан
3,'7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан
4,'9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан


In [ ]:
ind_order = {'Все товары':0, 'Горючие полезные ископаемые и первичная продукция из них':1, 'Уран, радиоактивные вещества и прочие изотопы':2, 'Электроэнергия':3,
             'Нерудные полезные ископаемые и первичная продукция из них':4, 'Рудные полезные ископаемые и первичная продукция из них':5, 'Драгоценные и полудрагоценыне камни и металлы и изделия из них':6,
             'Сырье для легкой промышленности (шкуры, кожа, мех, пряжа, ткани и пр.)':7, 'Продукция легкой промышленности':8, 'Изделия из древисины, включая бумажную продукцию':9,
             'Изделия из нерудных материалов (камень, кермика, стекло и пр.)':10, 'Продукция из пластмассы и резины, включая сырье':11, 'Фармацевтическая продукция':12,
             'Продукция химической промышленности':13, 'Изделия из металлов':14, 'Машины и оборудование (кроме транспортных)':15, 'Транспортные средства и оборудование':16,
             'Прочая промышленная непродовольственная продукция':17, 'Мясо-молочная и рыбная продукция, включая сырье':18, 'Зерновые и бобовые, продукция из них':19,
             'Плодоовощная продукция, включая сырье':20, 'Прочие продовольственыне товары и сырье для них':21, 'Прочее':22, 'Товары нигде не указанные':23}

In [ ]:
def change_colnames(df):
  num_of_years = len(df._get_numeric_data().columns)
  begin_year = 2023- num_of_years
  col_names_goods = list(range(begin_year,2023))
  col_names_goods.insert(0,'Код')
  col_names_goods.insert(1,'Наименование товара')
  col_names_goods.insert(8,'Направление')
  col_names_goods.insert(9,'Группа стран')
  col_names_goods.insert(10,'Страна')
  df = df.fillna(0)
  df.columns = col_names_goods
  return df


df_countries = change_colnames(df_countries)
df_countries.head()

,Код,Наименование товара,2017,2018,2019,2020,2021,2022,Направление,Группа стран,Страна
0,'7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан
1,'5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан
2,'2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан
3,'7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан
4,'9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан


In [ ]:
def remove_symbol(s):
  return s.replace("'","")

In [ ]:
df_countries['Код'] = df_countries['Код'].apply(remove_symbol)
df_goods['Код'] = df_goods['Код'].apply(remove_symbol)

df_countries['2017_2022'] = df_countries[2017] + df_countries[2018] + df_countries[2019] + df_countries[2020] + df_countries[2021] + df_countries[2022]
df_countries.head()


,Код,Наименование товара,2017,2018,2019,2020,2021,2022,Направление,Группа стран,Страна,2017_2022
0,7108,Золото (включая золото с гальваническим покрыт...,0.00,0.00,0.00,5804440.00,4109750.00,4110300.00,Экспорт,Все страны,Узбекистан,14024490.00
1,5205,"Пряжа хлопчатобумажная (кроме швейных ниток), ...",629003.00,722601.00,919771.00,933999.00,1603299.00,1390651.00,Экспорт,Все страны,Узбекистан,6199324.00
2,2711,Газы нефтяные и углеводороды газообразные прочие,1398875.00,2453903.00,2296084.00,478140.00,717324.00,925318.00,Экспорт,Все страны,Узбекистан,8269644.00
3,7403,Медь рафинированная и сплавы медные необработа...,320489.00,508686.00,560056.00,580697.00,737973.00,581528.00,Экспорт,Все страны,Узбекистан,3289429.00
4,9999,"Товары, нигде не указанные",3645206.00,3237880.00,5404450.00,544624.00,322670.00,319800.00,Экспорт,Все страны,Узбекистан,13474630.00


In [ ]:
def merge_columns_groups (df, good_groups, ind_order):
  good_groups = good_groups.drop('Наименование товара', axis=1, inplace = True)
  df_grouped = df.merge(df_goods, how = 'left', on = 'Код').fillna(0)
  grouped_column = df_grouped.pop('Отрасль')
  df_grouped.insert(loc=1, column='Отрасль', value=grouped_column)
  df_grouped = df_grouped.sort_values(by='Код', ascending = True)
  return df_grouped

In [ ]:
df_group = merge_columns_groups (df_countries, df_goods, ind_order).reset_index(drop=True)
df_group.head()

,Код,Отрасль,Наименование товара,2017,2018,2019,2020,2021,2022,Направление,Группа стран,Страна,2017_2022
0,0101,"Мясо-молочная и рыбная продукция, включая сырье","Лошади, ослы, мулы и лошаки живые",0.00,0.00,0.00,0.00,0.00,0.00,Экспорт,Европа,Таджикистан,0.00
1,0101,"Мясо-молочная и рыбная продукция, включая сырье","Лошади, ослы, мулы и лошаки живые",12.00,0.00,0.00,0.00,2.00,0.00,Импорт,ЦА,Туркменистан,14.00
2,0101,"Мясо-молочная и рыбная продукция, включая сырье","Лошади, ослы, мулы и лошаки живые",0.00,0.00,0.00,0.00,0.00,0.00,Экспорт,Европа,Туркменистан,0.00
3,0101,"Мясо-молочная и рыбная продукция, включая сырье","Лошади, ослы, мулы и лошаки живые",0.00,0.00,0.00,0.00,0.00,0.00,Экспорт,Восточная Азия кроме Китая,Узбекистан,0.00
4,0101,"Мясо-молочная и рыбная продукция, включая сырье","Лошади, ослы, мулы и лошаки живые",0.00,0.00,0.00,0.00,0.00,0.00,Импорт,Турция,Таджикистан,0.00


In [ ]:
direction = df_group['Направление'].unique().tolist()
direction

['Экспорт', 'Импорт']

In [ ]:
states = df_group['Страна'].unique().tolist()
states

['Таджикистан', 'Туркменистан', 'Узбекистан', 'Казахстан', 'Кыргызстан']

In [ ]:
## добавить все

In [ ]:
def get_all_CA(df, direction):
  df_1 = df_group.loc[(df_group['Направление']==direction)]
  df_1 = df_1.pivot_table(values='2017_2022', index=['Отрасль'], columns=['Группа стран'], aggfunc='sum')
  df_1 = df_1.reset_index()
  df_1['Другие_страны'] = df_1['Все страны'] - df_1['ЦА'] - df_1['Россия и Беларусь'] - df_1['Китай'] - df_1['Европа'] - df_1['Восточная Азия кроме Китая'] - df_1['Турция']
  df_1 = df_1[['Отрасль', 'Все страны', 'ЦА', 'Россия и Беларусь', 'Китай', 'Европа','Восточная Азия кроме Китая','Турция', 'Другие_страны']]

  row_all = df_1.loc[df_1.index[-1]].to_frame().T
  df_1.loc['Отрасль']= df_1.sum(numeric_only=True)
  df_1 = df_1.reset_index(drop=True)
  df_1.iloc[-1, df_1.columns.get_loc('Отрасль')] = 'Все товары'
  row_all = df_1.loc[df_1.index[-1]].to_frame().T
  df_1.drop(df_1.tail(1).index, inplace=True)
  df_2 = pd.concat([row_all, df_1], axis=0).reset_index(drop=True)
  df_2 = df_2.sort_values(by='Отрасль',  key=lambda x: x.map(ind_order)).reset_index()
  return df_2


In [ ]:
def get_data_by_countries_products(df_group,  direction, state):
  df_1 = df_group.loc[(df_group['Направление']==direction) & (df_group['Страна']==state)]
  if len(df_1)>0:
    df_industry = df_1.pivot_table(values='2017_2022', index=['Отрасль'], columns=['Группа стран'], aggfunc='sum').reset_index()
    df_industry = df_industry[['Отрасль', 'Все страны', 'ЦА', 'Россия и Беларусь', 'Китай', 'Европа','Восточная Азия кроме Китая','Турция']]
    df_industry['Другие_страны'] = df_industry['Все страны'] - df_industry['ЦА'] - df_industry['Россия и Беларусь'] - df_industry['Китай'] - df_industry['Европа'] - df_industry['Восточная Азия кроме Китая'] - df_industry['Турция']
    df_industry = df_industry.sort_values(by='Отрасль',  key=lambda x: x.map(ind_order)).reset_index(drop=True)
    row_all = df_industry.loc[df_industry.index[-1]].to_frame().T
    df_industry.loc['Отрасль']= df_industry.sum(numeric_only=True)
    df_industry = df_industry.reset_index(drop=True)
    df_industry.iloc[-1, df_industry.columns.get_loc('Отрасль')] = 'Все товары'
    row_all =df_industry.loc[df_industry.index[-1]].to_frame().T
    df_industry.drop(df_industry.tail(1).index, inplace=True)
    df_2 = pd.concat([row_all, df_industry], axis=0).reset_index(drop=True)
    return df_2



In [ ]:
with pd.ExcelWriter("/content/drive/MyDrive/Trade_CA/ЦА/Страны_товары/Анализ_страны_товары.xlsx") as writer:
  df_group.to_excel(writer, sheet_name='Все', index=False)

  for dir in direction:
    sht_name = f"{dir}_ЦА"
    print(f'{dir}')
    df_CA = get_all_CA(df_group, dir)
    df_CA.to_excel(writer, sheet_name=sht_name, index=False)



  for dir in direction:
    for state in states:
      print(f'{dir}_{state}')
      sht_name = f"{dir}_{state}"
      df_state_ind = get_data_by_countries_products(df_group, dir, state)
      df_state_ind.to_excel(writer, sheet_name=sht_name, index=False)

Экспорт
Импорт
Экспорт_Таджикистан
Экспорт_Туркменистан
Экспорт_Узбекистан
Экспорт_Казахстан
Экспорт_Кыргызстан
Импорт_Таджикистан
Импорт_Туркменистан
Импорт_Узбекистан
Импорт_Казахстан
Импорт_Кыргызстан


## Не вошедшее

In [ ]:
# def calclulate_percent(df):
#   df_change = df.select_dtypes(include=['int','float']).pct_change(axis=1).replace(np.inf, 1).fillna(0)
#   num_of_years = len(df._get_numeric_data().columns)
#   begin_year = 2023- num_of_years
#   #df_change[begin_year].replace(0, 1, inplace=True)
#   df_change += 1
#   df_change_cumproded = df_change.cumprod(axis=1)
#   df_change_cumproded.insert(loc=0, column='Отрасль', value=df['Отрасль'])
#   return  df_change_cumproded


# calclulate_percent(df_industry_by_years)

In [ ]:
!pip install world_trade_data --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for world_trade_data: filename=world_trade_data-0.1.1-py3-none-any.whl size=8834 sha256=a3cacf0977f0490d8ff9e8249b0e38237c76729e776f09207f4f1a9222c641df
  Stored in directory: /root/.cache/pip/wheels/6a/86/0c/70e52c532ae5904ab9b8640b1c6f1cd5029ad7679c8487ed57
Successfully built world_trade_data


In [ ]:
import world_trade_data as wits

In [ ]:
wits.get_countries()


,name,notes,countrycode,isreporter,ispartner,isgroup,grouptype
iso3Code,,,,,,,
AFG,Afghanistan,,004,False,False,No,N/A
ALB,Albania,,008,False,False,No,N/A
DZA,Algeria,,012,False,False,No,N/A
ASM,American Samoa,,016,True,False,No,N/A
AND,Andorra,,020,False,False,No,N/A
...,...,...,...,...,...,...,...
YEM,Yemen,,887,False,False,No,N/A
YDR,Yemen Democratic,,720,True,False,No,N/A
YUG,"Yugoslavia,FR(Serbia/Montenegr",,890,True,False,No,N/A


In [ ]:
wits.get_products()


,isgroup,nomenclaturecode,grouptype,productdescription,notes
productcode,,,,,
999999,N/A,N/A,N/A,Not Applicable,
01-05_Animal,Yes,H0,Sector,Animal,
06-15_Vegetable,Yes,H0,Sector,Vegetable,
16-24_FoodProd,Yes,H0,Sector,Food Products,
25-26_Minerals,Yes,H0,Sector,Minerals,
27-27_Fuels,Yes,H0,Sector,Fuels,
28-38_Chemicals,Yes,H0,Sector,Chemicals,
39-40_PlastiRub,Yes,H0,Sector,Plastic or Rubber,
41-43_HidesSkin,Yes,H0,Sector,Hides and Skins,


In [ ]:
wits.get_indicators()


,ispartnerequired,SDMX_partnervalue,isproductrequired,SDMX_productvalue,name,definition,source,topic,productclassification,periodicity,valuation,currency,notes
indicatorcode,,,,,,,,,,,,,
CNTRY-GRWTH,yes,,yes,,Country Growth (%),Annual percentage growth rate of the country’s...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,Export - FOB; Import - CIF,None,1) Mirror Exports is considered 2)Growth for a...
HH-MKT-CNCNTRTN-NDX,no,999,no,999999,HH Market concentration index,Hirschman Herfindahl index is a measure of th...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,Export - FOB; Import - CIF,None,1) Mirror Exports is considered for export dat...
MPRT-PRDCT-SHR,yes,,yes,,Product Share (%),The share of total merchandise trade (export o...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,None,None,
MPRT-PRTNR-SHR,yes,,no,999999,Partner Share (%),The share of total merchandise trade (export o...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,None,None,
MPRT-SHR-TTL-PRDCT,yes,,yes,,Share in Total Products (%),Share in Total Products (%),WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,None,None,1) All traded products at HS 6 digits are con...
MPRT-TRD-VL,yes,,yes,,Trade Value (US$ Thousand),Total Import/Export Value in thousands of US D...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,Export - FOB; Import - CIF,US Dollar - Current value,
NDX-XPRT-MKT-PNRTTN,no,999,no,999999,Index Of Export Market Penetration,It is calculated as the number of countries to...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,Export - FOB; Import - CIF,None,1) Mirror Exports is considered for export dat...
NMBR-MPRT-HS6-PRDCT,yes,,no,999999,No Of traded HS6 digit Products,Total number of products imported by a country...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,None,None,1) All traded products at HS 6 digits are con...
NMBR-MPRT-PRTNR,no,999,no,999999,Number of import partners,Number of countries from which a particular co...,WITS - UNSD Comtrade,Trade,"Harmonized System 1988/92, SITC Revision 2",Annual,None,None,1) Product 'Total' is used for this computation.


In [ ]:
wits.get_indicator('MPRT-TRD-VL', reporter='usa', year='2017', name_or_id='id')


Value  \
Freq Reporter Partner ProductCode     Indicator   Year                 
A    USA      WLD     01-05_Animal    MPRT-TRD-VL 2017   32315600.71   
                      06-15_Vegetable MPRT-TRD-VL 2017   55433914.46   
                      16-24_FoodProd  MPRT-TRD-VL 2017   65905989.81   
                      25-26_Minerals  MPRT-TRD-VL 2017    7583189.87   
                      27-27_Fuels     MPRT-TRD-VL 2017  204019056.86   
                      28-38_Chemicals MPRT-TRD-VL 2017  199687863.31   
                      39-40_PlastiRub MPRT-TRD-VL 2017   82974736.05   
                      41-43_HidesSkin MPRT-TRD-VL 2017   14520614.88   
                      44-49_Wood      MPRT-TRD-VL 2017   47394503.54   
                      50-63_TextCloth MPRT-TRD-VL 2017  114038540.90   
                      64-67_Footwear  MPRT-TRD-VL 2017   32040188.80   
                      68-71_StoneGlas MPRT-TRD-VL 2017   80706049.87   
                      72-83_Metals    MPRT-TRD-VL 2017  133293092.22   
                      84-85_MachElec  MPRT-TRD-VL 2017  700766968.09   
                      86-89_Transport MPRT-TRD-VL 2017  329451465.86   
                      90-99_Miscellan MPRT-TRD-VL 2017  305144851.42   
                      AgrRaw          MPRT-TRD-VL 2017   20336562.05   
                      Chemical        MPRT-TRD-VL 2017  221897814.06   
                      Food            MPRT-TRD-VL 2017  146178109.53   
                      Fuels           MPRT-TRD-VL 2017  204085129.65   
                      manuf           MPRT-TRD-VL 2017 1874163160.85   
                      OresMtls        MPRT-TRD-VL 2017   53736740.15   
                      Textiles        MPRT-TRD-VL 2017  122803815.47   
                      Total           MPRT-TRD-VL 2017 2405276626.66   
                      Transp          MPRT-TRD-VL 2017 1061446067.53   
                      UNCTAD-SoP1     MPRT-TRD-VL 2017  227562196.76   
                      UNCTAD-SoP2     MPRT-TRD-VL 2017  356603459.15   
                      UNCTAD-SoP3     MPRT-TRD-VL 2017  885215064.48   
                      UNCTAD-SoP4     MPRT-TRD-VL 2017  835944760.94   

                                                       DataSource  
Freq Reporter Partner ProductCode     Indicator   Year             
A    USA      WLD     01-05_Animal    MPRT-TRD-VL 2017   WITS-CMT  
                      06-15_Vegetable MPRT-TRD-VL 2017   WITS-CMT  
                      16-24_FoodProd  MPRT-TRD-VL 2017   WITS-CMT  
                      25-26_Minerals  MPRT-TRD-VL 2017   WITS-CMT  
                      27-27_Fuels     MPRT-TRD-VL 2017   WITS-CMT  
                      28-38_Chemicals MPRT-TRD-VL 2017   WITS-CMT  
                      39-40_PlastiRub MPRT-TRD-VL 2017   WITS-CMT  
                      41-43_HidesSkin MPRT-TRD-VL 2017   WITS-CMT  
                      44-49_Wood      MPRT-TRD-VL 2017   WITS-CMT  
                      50-63_TextCloth MPRT-TRD-VL 2017   WITS-CMT  
                      64-67_Footwear  MPRT-TRD-VL 2017   WITS-CMT  
                      68-71_StoneGlas MPRT-TRD-VL 2017   WITS-CMT  
                      72-83_Metals    MPRT-TRD-VL 2017   WITS-CMT  
                      84-85_MachElec  MPRT-TRD-VL 2017   WITS-CMT  
                      86-89_Transport MPRT-TRD-VL 2017   WITS-CMT  
                      90-99_Miscellan MPRT-TRD-VL 2017   WITS-CMT  
                      AgrRaw          MPRT-TRD-VL 2017   WITS-CMT  
                      Chemical        MPRT-TRD-VL 2017   WITS-CMT  
                      Food            MPRT-TRD-VL 2017   WITS-CMT  
                      Fuels           MPRT-TRD-VL 2017   WITS-CMT  
                      manuf           MPRT-TRD-VL 2017   WITS-CMT  
                      OresMtls        MPRT-TRD-VL 2017   WITS-CMT  
                      Textiles        MPRT-TRD-VL 2017   WITS-CMT  
                      Total           MPRT-TRD-VL 2017   WITS-CMT  
                      Transp          MPRT-TRD-VL 2017   WITS-